# 📘 Day 3: Deep Learning Project - Fashion Item Classifier

**🎯 Goal:** Build a complete end-to-end deep learning project from scratch

**⏱️ Time:** 90-120 minutes

**🌟 Why This Matters for AI:**
- Learn the COMPLETE workflow: data → training → deployment
- Build a production-ready image classifier
- Master best practices for real-world AI projects
- Portfolio project you can showcase to employers

**🔥 Real-World Applications:**
- E-commerce: Automatic product categorization
- Fashion retail: Visual search and recommendations
- Inventory management: Automated item classification
- Quality control: Defect detection in manufacturing

**🎯 Project Overview:**
- Dataset: Fashion-MNIST (70,000 grayscale images of clothing)
- Task: Classify 10 fashion categories
- Both TensorFlow and PyTorch implementations
- Complete pipeline: preprocessing → training → evaluation → deployment

---

## 📋 Project Roadmap

**Phase 1: Data Loading & Exploration**
- Load Fashion-MNIST dataset
- Visualize samples
- Understand class distribution

**Phase 2: Data Preprocessing**
- Normalize images
- Reshape for CNN input
- Split train/validation/test sets
- Create data loaders

**Phase 3: Model Architecture**
- Design CNN architecture
- Implement in both TensorFlow and PyTorch
- Compare approaches

**Phase 4: Training**
- Set up training loop
- Monitor metrics
- Implement early stopping
- Visualize training progress

**Phase 5: Evaluation**
- Test set performance
- Confusion matrix
- Per-class accuracy
- Error analysis

**Phase 6: Model Deployment**
- Save models
- Load for inference
- Make predictions on new images
- Export for production

---

## 📦 Setup & Installation

In [ ]:
# Install required libraries
!pip install tensorflow torch torchvision numpy matplotlib scikit-learn seaborn

# Import TensorFlow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Import PyTorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms

# Import utilities
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import time

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set random seeds
tf.random.set_seed(42)
torch.manual_seed(42)
np.random.seed(42)

# Check versions
print("="*60)
print("Environment Setup")
print("="*60)
print(f"TensorFlow: {tf.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print("="*60)

## 📊 Phase 1: Data Loading & Exploration

**Fashion-MNIST Dataset:**
- 70,000 grayscale images (60,000 train + 10,000 test)
- Size: 28x28 pixels
- 10 classes of fashion items
- Created by Zalando Research
- Drop-in replacement for MNIST

**Classes:**
0. T-shirt/top
1. Trouser
2. Pullover
3. Dress
4. Coat
5. Sandal
6. Shirt
7. Sneaker
8. Bag
9. Ankle boot

---

In [ ]:
# Load Fashion-MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

print("Dataset Information:")
print("="*60)
print(f"Training set: {X_train_full.shape}")
print(f"Test set: {X_test.shape}")
print(f"Image shape: {X_train_full[0].shape}")
print(f"Number of classes: {len(np.unique(y_train_full))}")
print(f"Pixel value range: [{X_train_full.min()}, {X_train_full.max()}]")
print("="*60)

# Class names
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print("\nClasses:")
for i, name in enumerate(class_names):
    print(f"{i}: {name}")

### 📈 Visualize Sample Images

In [ ]:
# Visualize random samples from each class
plt.figure(figsize=(15, 10))
for i in range(10):
    # Get random sample from class i
    idx = np.where(y_train_full == i)[0]
    random_idx = np.random.choice(idx, 5, replace=False)
    
    for j, sample_idx in enumerate(random_idx):
        plt.subplot(10, 5, i*5 + j + 1)
        plt.imshow(X_train_full[sample_idx], cmap='gray')
        if j == 0:
            plt.ylabel(class_names[i], fontsize=12, rotation=0, 
                      ha='right', va='center')
        plt.axis('off')

plt.suptitle('Fashion-MNIST: 5 Random Samples per Class', fontsize=16, y=0.995)
plt.tight_layout()
plt.show()

### 📊 Class Distribution Analysis

In [ ]:
# Analyze class distribution
unique, counts = np.unique(y_train_full, return_counts=True)

plt.figure(figsize=(12, 5))

# Bar plot
plt.subplot(1, 2, 1)
bars = plt.bar([class_names[i] for i in unique], counts, color='steelblue', edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Class Distribution in Training Set')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Add counts on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=9)

# Pie chart
plt.subplot(1, 2, 2)
plt.pie(counts, labels=[class_names[i] for i in unique], autopct='%1.1f%%', startangle=90)
plt.title('Class Distribution (%)')
plt.axis('equal')

plt.tight_layout()
plt.show()

# Check if balanced
is_balanced = np.std(counts) < 100
print(f"\n{'✅' if is_balanced else '⚠️'} Dataset is {'balanced' if is_balanced else 'imbalanced'}")
print(f"Standard deviation: {np.std(counts):.2f} samples")

### 🔍 Pixel Value Analysis

In [ ]:
# Analyze pixel value distribution
plt.figure(figsize=(15, 4))

# Sample image
plt.subplot(1, 3, 1)
sample_idx = 42
plt.imshow(X_train_full[sample_idx], cmap='gray')
plt.title(f'Sample Image: {class_names[y_train_full[sample_idx]]}')
plt.axis('off')

# Histogram of pixel values
plt.subplot(1, 3, 2)
plt.hist(X_train_full[sample_idx].flatten(), bins=50, color='steelblue', edgecolor='black')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.title('Pixel Value Distribution')
plt.grid(alpha=0.3)

# Overall statistics
plt.subplot(1, 3, 3)
stats_text = f"""
Dataset Statistics:
───────────────────
Mean: {X_train_full.mean():.2f}
Std: {X_train_full.std():.2f}
Min: {X_train_full.min()}
Max: {X_train_full.max()}

Shape: {X_train_full.shape}
Dtype: {X_train_full.dtype}
"""
plt.text(0.1, 0.5, stats_text, fontsize=12, family='monospace',
         verticalalignment='center')
plt.axis('off')

plt.tight_layout()
plt.show()

## 🔧 Phase 2: Data Preprocessing

**Preprocessing Steps:**
1. ✅ Normalize pixel values to [0, 1]
2. ✅ Reshape for CNN input (add channel dimension)
3. ✅ Split into train/validation sets
4. ✅ Convert labels to one-hot encoding (TensorFlow)
5. ✅ Create data loaders (PyTorch)

**Why Normalize?**
- Faster convergence
- Better gradient flow
- Prevents numerical instability

---

In [ ]:
# Split into train and validation sets
X_train = X_train_full[:-10000]
X_val = X_train_full[-10000:]
y_train = y_train_full[:-10000]
y_val = y_train_full[-10000:]

print("Data Split:")
print("="*60)
print(f"Training: {X_train.shape[0]:,} samples")
print(f"Validation: {X_val.shape[0]:,} samples")
print(f"Test: {X_test.shape[0]:,} samples")
print("="*60)

### 🔵 TensorFlow Preprocessing

In [ ]:
# TensorFlow preprocessing
print("\n🔵 TensorFlow Preprocessing:")
print("="*60)

# Normalize to [0, 1]
X_train_tf = X_train.astype('float32') / 255.0
X_val_tf = X_val.astype('float32') / 255.0
X_test_tf = X_test.astype('float32') / 255.0

# Reshape for CNN (add channel dimension)
X_train_tf = X_train_tf.reshape(-1, 28, 28, 1)
X_val_tf = X_val_tf.reshape(-1, 28, 28, 1)
X_test_tf = X_test_tf.reshape(-1, 28, 28, 1)

# One-hot encode labels
y_train_tf = to_categorical(y_train, 10)
y_val_tf = to_categorical(y_val, 10)
y_test_tf = to_categorical(y_test, 10)

print(f"✅ Input shape: {X_train_tf.shape}")
print(f"✅ Label shape: {y_train_tf.shape}")
print(f"✅ Pixel range: [{X_train_tf.min()}, {X_train_tf.max()}]")
print("="*60)

### 🔥 PyTorch Preprocessing

In [ ]:
# PyTorch preprocessing
print("\n🔥 PyTorch Preprocessing:")
print("="*60)

# Normalize to [0, 1]
X_train_pt = torch.FloatTensor(X_train) / 255.0
X_val_pt = torch.FloatTensor(X_val) / 255.0
X_test_pt = torch.FloatTensor(X_test) / 255.0

# Reshape for CNN (PyTorch uses channels-first: N, C, H, W)
X_train_pt = X_train_pt.unsqueeze(1)  # Add channel dimension
X_val_pt = X_val_pt.unsqueeze(1)
X_test_pt = X_test_pt.unsqueeze(1)

# Convert labels to tensors
y_train_pt = torch.LongTensor(y_train)
y_val_pt = torch.LongTensor(y_val)
y_test_pt = torch.LongTensor(y_test)

# Create datasets
train_dataset = TensorDataset(X_train_pt, y_train_pt)
val_dataset = TensorDataset(X_val_pt, y_val_pt)
test_dataset = TensorDataset(X_test_pt, y_test_pt)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"✅ Input shape: {X_train_pt.shape}")
print(f"✅ Label shape: {y_train_pt.shape}")
print(f"✅ Pixel range: [{X_train_pt.min()}, {X_train_pt.max()}]")
print(f"✅ Batch size: 128")
print(f"✅ Number of batches: {len(train_loader)}")
print("="*60)

## 🏗️ Phase 3: Model Architecture

**CNN Architecture Design:**
- 3 Convolutional blocks (each with Conv2D + MaxPooling)
- Batch Normalization for faster training
- Dropout for regularization
- Dense layers for classification

**Architecture:**
```
Input (28x28x1)
    ↓
Conv2D(32) + ReLU + BatchNorm + MaxPool → (14x14x32)
    ↓
Conv2D(64) + ReLU + BatchNorm + MaxPool → (7x7x64)
    ↓
Conv2D(128) + ReLU + BatchNorm + MaxPool → (3x3x128)
    ↓
Flatten → 1152
    ↓
Dense(256) + ReLU + Dropout(0.5)
    ↓
Dense(128) + ReLU + Dropout(0.5)
    ↓
Dense(10) + Softmax
```

---

### 🔵 TensorFlow Model

In [ ]:
# Build TensorFlow/Keras model
def create_tensorflow_model():
    model = models.Sequential([
        # Input layer
        layers.Input(shape=(28, 28, 1)),
        
        # Convolutional Block 1
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Convolutional Block 2
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Convolutional Block 3
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten
        layers.Flatten(),
        
        # Dense layers
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        
        # Output layer
        layers.Dense(10, activation='softmax')
    ], name='FashionCNN_TensorFlow')
    
    return model

# Create model
tf_model = create_tensorflow_model()

# Compile model
tf_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("🔵 TensorFlow Model:")
print("="*60)
tf_model.summary()
print("="*60)

### 🔥 PyTorch Model

In [ ]:
# Build PyTorch model
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        
        # Convolutional Block 1
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Convolutional Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Convolutional Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        # Dense layers
        self.fc1 = nn.Linear(128 * 3 * 3, 256)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 10)
        
    def forward(self, x):
        # Conv Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        
        # Conv Block 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        
        # Conv Block 3
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Dense layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        
        return x

# Create model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pt_model = FashionCNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pt_model.parameters(), lr=0.001)

print("\n🔥 PyTorch Model:")
print("="*60)
print(pt_model)
print("="*60)
print(f"Total parameters: {sum(p.numel() for p in pt_model.parameters()):,}")
print(f"Device: {device}")
print("="*60)

## 🚀 Phase 4: Training

**Training Configuration:**
- Optimizer: Adam
- Learning rate: 0.001
- Batch size: 128
- Epochs: 20 (with early stopping)
- Early stopping patience: 3 epochs

---

### 🔵 Train TensorFlow Model

In [ ]:
# Setup callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

model_checkpoint = ModelCheckpoint(
    'best_fashion_model_tf.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Train model
print("🚀 Training TensorFlow Model...\n")
start_time = time.time()

tf_history = tf_model.fit(
    X_train_tf, y_train_tf,
    validation_data=(X_val_tf, y_val_tf),
    epochs=20,
    batch_size=128,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

tf_training_time = time.time() - start_time
print(f"\n✅ Training complete in {tf_training_time:.2f} seconds")

### 🔥 Train PyTorch Model

In [ ]:
# Training function
def train_pytorch_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, patience=3):
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track metrics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += batch_y.size(0)
            train_correct += (predicted == batch_y).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += batch_y.size(0)
                val_correct += (predicted == batch_y).sum().item()
        
        # Calculate averages
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_acc = 100 * train_correct / train_total
        val_acc = 100 * val_correct / val_total
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"Epoch [{epoch+1}/{num_epochs}] - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_fashion_model_pt.pth')
            print("✅ Model saved (best validation loss)")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered after {epoch+1} epochs")
                break
    
    return history

# Train model
print("🚀 Training PyTorch Model...\n")
start_time = time.time()

pt_history = train_pytorch_model(
    pt_model, train_loader, val_loader,
    criterion, optimizer,
    num_epochs=20, patience=3
)

pt_training_time = time.time() - start_time
print(f"\n✅ Training complete in {pt_training_time:.2f} seconds")

### 📊 Training Progress Visualization

In [ ]:
# Visualize training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# TensorFlow - Loss
axes[0, 0].plot(tf_history.history['loss'], label='Train Loss', marker='o')
axes[0, 0].plot(tf_history.history['val_loss'], label='Val Loss', marker='o')
axes[0, 0].set_title('TensorFlow: Training & Validation Loss', fontsize=14)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# TensorFlow - Accuracy
axes[0, 1].plot(tf_history.history['accuracy'], label='Train Accuracy', marker='o')
axes[0, 1].plot(tf_history.history['val_accuracy'], label='Val Accuracy', marker='o')
axes[0, 1].set_title('TensorFlow: Training & Validation Accuracy', fontsize=14)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# PyTorch - Loss
axes[1, 0].plot(pt_history['train_loss'], label='Train Loss', marker='o')
axes[1, 0].plot(pt_history['val_loss'], label='Val Loss', marker='o')
axes[1, 0].set_title('PyTorch: Training & Validation Loss', fontsize=14)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# PyTorch - Accuracy
axes[1, 1].plot(pt_history['train_acc'], label='Train Accuracy', marker='o')
axes[1, 1].plot(pt_history['val_acc'], label='Val Accuracy', marker='o')
axes[1, 1].set_title('PyTorch: Training & Validation Accuracy', fontsize=14)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Accuracy (%)')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Training comparison
print("\n" + "="*60)
print("Training Comparison")
print("="*60)
print(f"TensorFlow training time: {tf_training_time:.2f} seconds")
print(f"PyTorch training time: {pt_training_time:.2f} seconds")
print(f"\nFinal TensorFlow validation accuracy: {tf_history.history['val_accuracy'][-1]*100:.2f}%")
print(f"Final PyTorch validation accuracy: {pt_history['val_acc'][-1]:.2f}%")
print("="*60)

## 📊 Phase 5: Evaluation

**Evaluation Metrics:**
- Test accuracy
- Per-class accuracy
- Confusion matrix
- Classification report (precision, recall, F1-score)
- Error analysis

---

### 🔵 Evaluate TensorFlow Model

In [ ]:
# Evaluate on test set
print("🔵 TensorFlow Model Evaluation:")
print("="*60)

tf_test_loss, tf_test_acc = tf_model.evaluate(X_test_tf, y_test_tf, verbose=0)
print(f"Test Loss: {tf_test_loss:.4f}")
print(f"Test Accuracy: {tf_test_acc*100:.2f}%")

# Get predictions
tf_predictions = tf_model.predict(X_test_tf, verbose=0)
tf_pred_classes = np.argmax(tf_predictions, axis=1)

print("="*60)

### 🔥 Evaluate PyTorch Model

In [ ]:
# Load best model
pt_model.load_state_dict(torch.load('best_fashion_model_pt.pth'))
pt_model.eval()

print("\n🔥 PyTorch Model Evaluation:")
print("="*60)

# Evaluate on test set
test_loss = 0.0
test_correct = 0
test_total = 0
pt_pred_classes = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = pt_model(batch_X)
        loss = criterion(outputs, batch_y)
        
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        test_total += batch_y.size(0)
        test_correct += (predicted == batch_y).sum().item()
        
        pt_pred_classes.extend(predicted.cpu().numpy())

pt_test_loss = test_loss / len(test_loader)
pt_test_acc = 100 * test_correct / test_total

print(f"Test Loss: {pt_test_loss:.4f}")
print(f"Test Accuracy: {pt_test_acc:.2f}%")
print("="*60)

### 📊 Confusion Matrix

In [ ]:
# Create confusion matrices
tf_cm = confusion_matrix(y_test, tf_pred_classes)
pt_cm = confusion_matrix(y_test, pt_pred_classes)

# Plot confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# TensorFlow confusion matrix
sns.heatmap(tf_cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0], cbar_kws={'label': 'Count'})
axes[0].set_title('TensorFlow: Confusion Matrix', fontsize=14)
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
plt.setp(axes[0].get_xticklabels(), rotation=45, ha='right')

# PyTorch confusion matrix
sns.heatmap(pt_cm, annot=True, fmt='d', cmap='Oranges',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[1], cbar_kws={'label': 'Count'})
axes[1].set_title('PyTorch: Confusion Matrix', fontsize=14)
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('True')
plt.setp(axes[1].get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

### 📈 Per-Class Performance

In [ ]:
# Classification reports
print("\n🔵 TensorFlow Classification Report:")
print("="*60)
print(classification_report(y_test, tf_pred_classes, target_names=class_names))

print("\n🔥 PyTorch Classification Report:")
print("="*60)
print(classification_report(y_test, pt_pred_classes, target_names=class_names))

In [ ]:
# Calculate per-class accuracy
tf_class_acc = tf_cm.diagonal() / tf_cm.sum(axis=1) * 100
pt_class_acc = pt_cm.diagonal() / pt_cm.sum(axis=1) * 100

# Plot per-class accuracy
x = np.arange(len(class_names))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 6))
bars1 = ax.bar(x - width/2, tf_class_acc, width, label='TensorFlow', color='steelblue')
bars2 = ax.bar(x + width/2, pt_class_acc, width, label='PyTorch', color='darkorange')

ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Per-Class Accuracy Comparison', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(class_names, rotation=45, ha='right')
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Print statistics
print("\nPer-Class Accuracy Statistics:")
print("="*60)
for i, name in enumerate(class_names):
    print(f"{name:15} - TF: {tf_class_acc[i]:5.2f}% | PT: {pt_class_acc[i]:5.2f}%")
print("="*60)

### 🔍 Error Analysis

In [ ]:
# Find misclassified examples (using TensorFlow predictions)
misclassified_idx = np.where(tf_pred_classes != y_test)[0]

print(f"Total misclassified: {len(misclassified_idx)} out of {len(y_test)} ({len(misclassified_idx)/len(y_test)*100:.2f}%)\n")

# Visualize some misclassified examples
plt.figure(figsize=(15, 8))
for i, idx in enumerate(misclassified_idx[:20]):
    plt.subplot(4, 5, i + 1)
    plt.imshow(X_test[idx], cmap='gray')
    
    true_label = class_names[y_test[idx]]
    pred_label = class_names[tf_pred_classes[idx]]
    confidence = tf_predictions[idx][tf_pred_classes[idx]] * 100
    
    plt.title(f"True: {true_label}\nPred: {pred_label}\n({confidence:.1f}%)",
              fontsize=9, color='red')
    plt.axis('off')

plt.suptitle('Misclassified Examples (TensorFlow)', fontsize=16, y=1.00)
plt.tight_layout()
plt.show()

## 💾 Phase 6: Model Deployment

**Deployment Steps:**
1. Save trained models
2. Create inference functions
3. Test on new images
4. Export for production

---

### 💾 Save Models

In [ ]:
# Save TensorFlow model
tf_model.save('fashion_classifier_tf.keras')
print("✅ TensorFlow model saved as 'fashion_classifier_tf.keras'")

# Save PyTorch model
torch.save({
    'model_state_dict': pt_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'class_names': class_names
}, 'fashion_classifier_pt.pth')
print("✅ PyTorch model saved as 'fashion_classifier_pt.pth'")

### 🔮 Inference Functions

In [ ]:
# TensorFlow inference function
def predict_tensorflow(image, model):
    """
    Predict fashion item class using TensorFlow model.
    
    Args:
        image: Input image (28x28 grayscale)
        model: Trained TensorFlow model
    
    Returns:
        predicted_class: Predicted class name
        confidence: Prediction confidence (0-1)
        probabilities: All class probabilities
    """
    # Preprocess
    if len(image.shape) == 2:
        image = image.reshape(1, 28, 28, 1)
    image = image.astype('float32') / 255.0
    
    # Predict
    predictions = model.predict(image, verbose=0)[0]
    predicted_idx = np.argmax(predictions)
    
    return class_names[predicted_idx], predictions[predicted_idx], predictions

# PyTorch inference function
def predict_pytorch(image, model):
    """
    Predict fashion item class using PyTorch model.
    
    Args:
        image: Input image (28x28 grayscale)
        model: Trained PyTorch model
    
    Returns:
        predicted_class: Predicted class name
        confidence: Prediction confidence (0-1)
        probabilities: All class probabilities
    """
    model.eval()
    
    # Preprocess
    if len(image.shape) == 2:
        image = torch.FloatTensor(image).unsqueeze(0).unsqueeze(0)
    image = image / 255.0
    image = image.to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)[0]
        predicted_idx = torch.argmax(probabilities).item()
    
    return class_names[predicted_idx], probabilities[predicted_idx].item(), probabilities.cpu().numpy()

print("✅ Inference functions created!")

### 🎯 Test Inference

In [ ]:
# Test on random samples
test_indices = np.random.choice(len(X_test), 10, replace=False)

plt.figure(figsize=(18, 10))
for i, idx in enumerate(test_indices):
    # Get predictions
    tf_pred, tf_conf, tf_probs = predict_tensorflow(X_test[idx], tf_model)
    pt_pred, pt_conf, pt_probs = predict_pytorch(X_test[idx], pt_model)
    
    # Plot image
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_test[idx], cmap='gray')
    
    true_label = class_names[y_test[idx]]
    color = 'green' if tf_pred == true_label else 'red'
    
    title = f"True: {true_label}\n"
    title += f"TF: {tf_pred} ({tf_conf*100:.1f}%)\n"
    title += f"PT: {pt_pred} ({pt_conf*100:.1f}%)"
    
    plt.title(title, fontsize=9, color=color)
    plt.axis('off')

plt.suptitle('Model Predictions Comparison', fontsize=16)
plt.tight_layout()
plt.show()

print("\n🎯 Green = Correct, Red = Incorrect")

## 🎯 Interactive Exercise 1: Improve the Model

**Challenge:** Can you beat the baseline models?

**Ideas to try:**
1. Add more convolutional layers
2. Increase filters (32 → 64 → 128 → 256)
3. Add data augmentation (rotation, flip, zoom)
4. Try different optimizers (SGD with momentum, RMSprop)
5. Adjust learning rate
6. Add more dense layers
7. Try different batch sizes
8. Implement learning rate scheduling

**Your Turn!** 👇

In [ ]:
# Exercise 1: Improve the model
# TODO: Build your improved model here

# Ideas:
# - Add data augmentation
# - Deeper network
# - Different architecture (ResNet-style, etc.)

# YOUR CODE HERE

## 🎯 Interactive Exercise 2: Build a Web App

**Challenge:** Create a simple web interface for your model

**Steps:**
1. Use Gradio or Streamlit
2. Load your trained model
3. Accept image uploads
4. Display predictions with confidence scores
5. Show top-3 predictions

**Starter Code:**

In [ ]:
# Exercise 2: Web App (optional)
# Uncomment and run:

# !pip install gradio
# import gradio as gr

# def classify_fashion_item(image):
#     # Preprocess image
#     # Make prediction
#     # Return result
#     pass

# interface = gr.Interface(
#     fn=classify_fashion_item,
#     inputs=gr.Image(shape=(28, 28)),
#     outputs="text",
#     title="Fashion Item Classifier"
# )

# interface.launch()

## 🎉 Congratulations!

**You just built a complete end-to-end deep learning project!**

**What you accomplished:**
- ✅ Loaded and explored a real-world dataset
- ✅ Preprocessed data for deep learning
- ✅ Built CNN models in BOTH TensorFlow and PyTorch
- ✅ Trained models with early stopping
- ✅ Evaluated performance with multiple metrics
- ✅ Created confusion matrices and error analysis
- ✅ Saved models for deployment
- ✅ Built inference functions
- ✅ Compared TensorFlow vs PyTorch

**🔥 Real-World Skills:**
- Production-ready image classification
- Complete ML pipeline development
- Model evaluation and debugging
- Framework comparison (TensorFlow vs PyTorch)
- Deployment preparation

**📊 Project Results:**
- Trained 2 high-accuracy fashion classifiers
- Achieved ~90%+ test accuracy
- Identified which classes are hardest to classify
- Ready-to-deploy models

**🎯 Next Steps:**
1. Deploy to cloud (AWS, GCP, Azure)
2. Create REST API with FastAPI
3. Build mobile app with TensorFlow Lite
4. Try transfer learning with pre-trained models
5. Experiment with other datasets (CIFAR-10, ImageNet)

**🔥 2024-2025 Applications:**
- E-commerce visual search
- Automated inventory management
- Fashion recommendation systems
- Quality control in manufacturing
- Multimodal AI (combine with text descriptions)

**💼 Portfolio Project:**
- Add to GitHub with README
- Deploy as web app
- Showcase in job interviews
- Write a blog post about it

---

**📚 What's Next:**
- Week 13: Transfer Learning & Pre-trained Models
- Week 14: Advanced CNN Architectures (ResNet, EfficientNet)
- Week 15: Recurrent Neural Networks & LSTMs
- Week 16: Transformers & Attention Mechanisms

**💬 Share Your Project:**
- Tweet your results with #100DaysOfCode
- Post on LinkedIn
- Share on GitHub
- Help others learn!

---

*Remember: You now have a complete deep learning project in your portfolio. This is EXACTLY what employers want to see!* 🚀

**Keep learning, keep building, keep shipping!** 💪